In [1]:
import os
import sys
import numpy as np
import argparse
from pyrite import _utils as ut
import glob
import matplotlib.pyplot as plt
from gigpy import elevation_models

from scipy.optimize import leastsq
from scipy.stats.stats import pearsonr

from pykrige import OrdinaryKriging

import matlab.engine
#### define dicts for model/residual/initial values ###############

def remove_ramp(lat,lon,data):
    # mod = a*x + b*y + c*x*y
    lat = lat/180*np.pi
    lon = lon/180*np.pi  
    lon = lon*np.cos(lat) # to get isometrics coordinates
    
    p0 = [0.0001,0.0001,0.0001,0.0000001]
    plsq = leastsq(residual_trend,p0,args = (lat,lon,data))
    para = plsq[0]
    data_trend = data - func_trend(lat,lon,para)
    corr, _ = pearsonr(data, func_trend(lat,lon,para))
    return data_trend, para, corr

def func_trend(lat,lon,p):
    a0,b0,c0,d0 = p
    return a0 + b0*lat + c0*lon +d0*lat*lon

def residual_trend(p,lat,lon,y0):
    a0,b0,c0,d0 = p 
    return y0 - func_trend(lat,lon,p)



model_dict = {'linear': elevation_models.linear_elevation_model,
                      'onn': elevation_models.onn_elevation_model,
                      'onn_linear': elevation_models.onn_linear_elevation_model,
                      'exp': elevation_models.exp_elevation_model,
                      'exp_linear': elevation_models.exp_linear_elevation_model}

residual_dict = {'linear': elevation_models.residuals_linear,
                      'onn': elevation_models.residuals_onn,
                      'onn_linear': elevation_models.residuals_onn_linear,
                      'exp': elevation_models.residuals_exp,
                      'exp_linear': elevation_models.residuals_exp_linear}

initial_dict = {'linear': elevation_models.initial_linear,
                      'onn': elevation_models.initial_onn,
                      'onn_linear': elevation_models.initial_onn_linear,
                      'exp': elevation_models.initial_exp,
                      'exp_linear': elevation_models.initial_exp_linear}

para_numb_dict = {'linear': 2,
                  'onn' : 3,
                  'onn_linear':4,
                  'exp':2,
                  'exp_linear':3}

font = {'family': 'serif',
        'color':  'black',
        'weight': 'bold',
        'size': 12,
        }

k0 = 10
cdic = ut.initconst()
raw_dir = '/Users/caoy0a/Documents/SCRATCH/LosAngelesT71F479D/pyrite/ERA5/raw'
era5_file = '/Users/caoy0a/Documents/SCRATCH/LosAngelesT71F479D/era5_sample.h5'
date_list = ut.read_hdf5(era5_file,datasetName = 'date')[0]
date_list = date_list.astype('U13')
date0 = date_list[k0]
print(date0)
fname0 = glob.glob(raw_dir + '/ERA*' + date0 + '*')[0]
print(fname0)
lvls,latlist,lonlist,gph,tmp,vpr = ut.get_ecmwf('ERA5',fname0,cdic, humidity='Q')
lonlist = lonlist - 360.0


# Make a height scale
hgt = np.linspace(cdic['minAltP'], gph.max().round(), cdic['nhgt'])
# Interpolate pressure, temperature and Humidity of hgt
[Pi,Ti,Vi] = ut.intP2H(lvls, hgt, gph, tmp, vpr, cdic)
# Calculate the delays
[DDry,DWet] = ut.PTV2del(Pi,Ti,Vi,hgt,cdic)

hgt0 = hgt.copy()
idx0 = np.where(((-100<hgt0) & (hgt0 < 5000)))
hgt00 = hgt0[idx0]

hgt0 = list(hgt0)
dd0 = DDry.copy()
dw0 = DWet.copy()
dt0 = dd0 + dw0

dd00 = dd0[:,:,idx0]
dw00 = dw0[:,:,idx0] 
dd00 = dd00.flatten()
dw00 = dw00.flatten()
dt00 = dd00 + dw00

print(len(hgt00))
nk = len(hgt00)

mdd = np.zeros((nk,),dtype = np.float32)
mdw = np.zeros((nk,),dtype = np.float32)
mdt = np.zeros((nk,),dtype = np.float32)

for i in range(nk):
    mdd[i] = np.mean(dd0[:,:,hgt0.index(hgt00[i])])
    mdw[i] = np.mean(dw0[:,:,hgt0.index(hgt00[i])])
    mdt[i] = np.mean(dt0[:,:,hgt0.index(hgt00[i])])

R = 6371    
BIN_NUMB = 55
max_length = 280
model = 'spherical'
range0 = np.asarray(1/2*max_length)
range0 = range0.tolist()

eng = matlab.engine.start_matlab()
k0 = 10
tzd = dt0[:,:,hgt0.index(hgt00[k0])]
lon = lonlist.flatten() 
lat = latlist.flatten() 
tzd = tzd.flatten()
tzd0, para, corr= remove_ramp(lat,lon,tzd)
uk = OrdinaryKriging(lon, lat, tzd0, coordinates_type = 'geographic', nlags=BIN_NUMB)
Semivariance_trend = 2*(uk.semivariance)   
x0 = (uk.lags)/180*np.pi*R
y0 = Semivariance_trend

LL0 = x0[x0< max_length]
SS0 = y0[x0< max_length]
sill0 = max(SS0)
sill0 = sill0.tolist()

LLm = matlab.double(LL0.tolist())
SSm = matlab.double(SS0.tolist()) 
tt = eng.variogramfit(LLm,SSm,range0,sill0,[],'nugget',0.00001,'model',model)
para = np.asarray(tt)
print(para)
eng.quit()

OSError: Unable to open file (unable to open file: name = '/Users/caoy0a/Documents/SCRATCH/LosAngelesT71F479D/era5_sample.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)